In [1]:
#imported necessary modules
import os
import zipfile
import pyttsx3
import random
import shutil
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from shutil import copyfile
from os import getcwd
from os import listdir
import cv2
import pandas as pd
import imutils
from tensorflow.keras.layers import Conv2D, Input, ZeroPadding2D, BatchNormalization, Activation, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
import matplotlib.image  as mpimg

###text to speech recognition 
def speak(text):
    language='en'
    engine=pyttsx3.init()
    engine.say(text)
    engine.runAndWait()
speak("Hi! Welcome to Our Project")

In [2]:
def data_desc(main_path):
    mask_path = main_path+'yesreal'
    unmask_path = main_path+'noreal'
    m_yes = len(listdir(mask_path))
    m_no = len(listdir(unmask_path))
    m_t = (m_yes+m_no)
    
    yes_pri = (m_yes* 100.0)/ m_t
    no_pri = (m_no* 100.0)/ m_t
    
    print(f"Total number of images in dataset: {m_t}")
    speak(f"Total number of images in dataset: {m_t}")
    print(f"Total Percentage of facemask images: {yes_pri}%, Total Number of facemask images: {m_yes}") 
    speak(f"Total Percentage of facemask images: {yes_pri}%, Total Number of facemask images: {m_yes}")
    print(f"Total Percentage of unmask images: {no_pri}%, Total Number of unmask images: {m_no}")
    speak(f"Total Percentage of unmask images: {no_pri}%, Total Number of unmask images: {m_no}") 
    
#description of training dataset
speak("Description of training dataset")
data_path = 'Dataset/'    
data_desc(data_path)

Total number of images in dataset: 4416
Total Percentage of facemask images: 55.41213768115942%, Total Number of facemask images: 2447
Total Percentage of unmask images: 44.58786231884058%, Total Number of unmask images: 1969


In [3]:
def split_data(SOURCE, TRAINING, TESTING, SPLIT_SIZE):
    dataset = []
    
    for unitData in os.listdir(SOURCE):
        data = SOURCE + unitData
        if(os.path.getsize(data) > 0):
            dataset.append(unitData)
        else:
            print('Skipped ' + unitData)
            print('Invalid file i.e zero size')
    
    train_set_length = int(len(dataset) * SPLIT_SIZE)
    test_set_length = int(len(dataset) - train_set_length)
    shuffled_set = random.sample(dataset, len(dataset))
    train_set = dataset[0:train_set_length]
    test_set = dataset[-test_set_length:]
       
    for unitData in train_set:
        temp_train_set = SOURCE + unitData
        final_train_set = TRAINING + unitData
        copyfile(temp_train_set, final_train_set)
    
    for unitData in test_set:
        temp_test_set = SOURCE + unitData
        final_test_set = TESTING + unitData
        copyfile(temp_test_set, final_test_set)
    speak("Data spliting Successfull")
YES_SOURCE_DIR = "Dataset/yesreal/"
TRAINING_YES_DIR = "Dataset/train/yes/"
TESTING_YES_DIR = "Dataset/test/yes/"
NO_SOURCE_DIR = "Dataset/noreal/"
TRAINING_NO_DIR = "Dataset/train/no/"
TESTING_NO_DIR = "Dataset/test/no/"

speak("Please wait! will data is being splitted to test and train directory")
split_size = .8
split_data(YES_SOURCE_DIR, TRAINING_YES_DIR, TESTING_YES_DIR, split_size)
split_data(NO_SOURCE_DIR, TRAINING_NO_DIR, TESTING_NO_DIR, split_size)
        

In [4]:
#description of training and testing directory
speak("Description of testing and training dataset after splitting")
train_yes=len(os.listdir('Dataset/train/yes'))
test_yes=len(os.listdir('Dataset/test/yes'))
train_no=len(os.listdir('Dataset/train/no'))
test_no=len(os.listdir('Dataset/test/no'))

print(f"The number of images with facemask in the train directory: {train_yes}")
speak(f"The number of images with facemask in the train directory: {train_yes}")
print(f"The number of images with facemask in the test directory: {test_yes}")
speak(f"The number of images with facemask in the test directory: {test_yes}")
print(f"The number of images without facemask in the train directory: {train_no}")
speak(f"The number of images without facemask in the train directory: {train_no}")
print(f"The number of images without facemask in the test directory: {test_no}")
speak(f"The number of images without facemask in the test directory: {test_no}")

The number of images with facemask in the train directory: 2447
The number of images with facemask in the test directory: 490
The number of images without facemask in the train directory: 1969
The number of images without facemask in the test directory: 394


In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(100, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Conv2D(100, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

In [6]:
TRAINING_DIR = "Dataset/train"
train_datagen = ImageDataGenerator(rescale=1.0/255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

train_generator = train_datagen.flow_from_directory(TRAINING_DIR, 
                                                    batch_size=10, 
                                                    target_size=(150, 150))
VALIDATION_DIR = "Dataset/test"
validation_datagen = ImageDataGenerator(rescale=1.0/255)

validation_generator = validation_datagen.flow_from_directory(VALIDATION_DIR, 
                                                         batch_size=10, 
                                                         target_size=(150, 150))
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')

Found 4416 images belonging to 2 classes.
Found 884 images belonging to 2 classes.


In [7]:
history = model.fit_generator(train_generator,
                              epochs=5,
                              validation_data=validation_generator,
                              callbacks=[checkpoint])

Instructions for updating:
Please use Model.fit, which supports generators.
  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 442 steps, validate for 89 steps
Epoch 1/5
 33/442 [=>............................] - ETA: 2:56 - loss: 0.6931 - acc: 0.6545

C:\Users\Amit\anaconda3\envs\tensorflow\lib\site-packages\PIL\Image.py:952: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  "Palette images with Transparency expressed in bytes should be "


441/442 [============================>.] - ETA: 0s - loss: 0.3967 - acc: 0.8441WARNING:tensorflow:From C:\Users\Amit\anaconda3\envs\tensorflow\lib\site-packages\tensorflow_core\python\ops\resource_variable_ops.py:1786: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: model-001.model\assets
442/442 [==============================] - 201s 454ms/step - loss: 0.3961 - acc: 0.8442 - val_loss: 0.2866 - val_acc: 0.8824
Epoch 2/5
442/442 [==============================] - 275s 623ms/step - loss: 0.3308 - acc: 0.8764 - val_loss: 0.2889 - val_acc: 0.8744
Epoch 3/5
442/442 [==============================] - 273s 618ms/step - loss: 0.3055 - acc: 0.8861 - val_loss: 0.2979 - val_acc: 0.8676
Epoch 4/5
442/442 [==============================] - 276s 623ms/step - loss: 0.3028 - a

In [10]:
color_dict={0:(0,255,0),1:(0,0,255)}
labels_dict={0:"Masked",1:"Unmasked"}
size = 4
speak("Please wait Your Camera is being open")
webcam = cv2.VideoCapture(0) #Use camera 0

# We load the xml file
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

busy0=0
busy1=0
while True:
    (rval, im) = webcam.read()
    im=cv2.flip(im,1,1) #Flip to act as a mirror

    # Resize the image to speed up detection
    mini = cv2.resize(im, (im.shape[1] // size, im.shape[0] // size))

    # detect MultiScale / faces 
    faces = face_classifier.detectMultiScale(mini)

    # Draw rectangles around each face
    for f in faces:
        (x, y, w, h) = [v * size for v in f] #Scale the shapesize backup
        #Save just the rectangle faces in SubRecFaces
        face_img = im[y:y+h, x:x+w]
        resized=cv2.resize(face_img,(150,150))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,150,150,3))
        reshaped = np.vstack([reshaped])
        result=model.predict(reshaped)
        #print(result)
        
        label=np.argmax(result,axis=1)[0]
        print(label)
        cv2.rectangle(im,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(im,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(im, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        flag=label
        if(label==0 and busy0==0):
            busy0=1
            busy1=0
            speak("Wearing Mask")
        if(label==1 and busy1==0):
            busy1=1
            busy0=0
            speak("Not Wearing Mask")
        
        
    # Show the image
    cv2.imshow('LIVE',   im)
    key = cv2.waitKey(10)
    # if Esc key is press then break out of the loop 
    if key == 27: #The Esc key
        break
# Stop video
webcam.release()

# Close all started windows
cv2.destroyAllWindows()

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
